In [39]:
import numpy as np
import math
import cmath

def generate_s_k(k, Dt):
    # use real hadmard test to generate data
    return np.array([1,2,3,4,5,6,7])

def make_X(first, last, k, d, s_k = []):
    X = np.zeros((k, d))
    for i in range(k):
        for j in range(d):
            X[i][j] = s_k[i+j-first]
    return X

def regression(X, noise_threshold):
    # print("X", X)
    U, sigma, V = np.linalg.svd(X)
    threshold = noise_threshold*max(sigma)
    # print("U", U)
    # print("sigma", sigma)
    # print("V", V)
    for l in range(len(sigma)):
        # print("iteration")
        if threshold <= sigma[l]:
            sigma[l] = 0
            U[:,l] = 0
            V[l,:] = 0
        # print("U", U)
        # print("sigma", sigma)
        # print("V", V)
    return U @ np.diag(sigma) @ V

'''
Dt: time step for each iteration (Dt should not be small)
noise_threshold: used for linear regression
err_threshold: how close the energy until iterations have been stopped
K: number of time steps to take each
alpha: used for calculating d  
'''
def ODMD(Dt, noise_threshold, err_threshold, K = 7, alpha = 1/2): 
    k = 0
    d = math.floor(alpha*(K+1))
    while (True):
        s_k = generate_s_k(k, Dt)
        X = make_X(0, K, d, d, s_k=s_k) 
        Xprime = make_X(1, K+1, d, d, s_k=s_k)
        X_sigma = regression(X, noise_threshold)
        A = Xprime @ np.linalg.pinv(X_sigma)
        k = k+1
        eigenvalues,_ = np.linalg.eig(A)
        eigenlog = [cmath.log(i) for i in eigenvalues]
        print(eigenlog)
        E_0 = -max(eigenlog, key=abs)/Dt
        print("E_0 =", E_0)
        try:
            if abs(old_E_0 - E_0) < err_threshold:
                break
        except:
            print("none")
        finally:
            old_E_0 = E_0
        
    return E_0

ODMD(.2, .0001, 0.001)

[(35.74685857892048+0j), (1.8972909199074663+2.41552069849701j), (1.8972909199074663-2.41552069849701j), (1.498364322632206+0j)]
E_0 = (-178.7342928946024+0j)
none
[(35.74685857892048+0j), (1.8972909199074663+2.41552069849701j), (1.8972909199074663-2.41552069849701j), (1.498364322632206+0j)]
E_0 = (-178.7342928946024+0j)


(-178.7342928946024+0j)